#[Exp-06]NLP_Task1 : 작사작곡

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 목차

project1: 멋진 작사가 만들기 
- 1.데이터 다운로드
- 2.데이터 읽어오기
- 3.데이터 정제
- 4.평가 데이터셋 분리
- 5.인공지능 만들기 


회고

# 1.라이브러리 버전 확인

In [2]:
import tensorflow

print(tensorflow.__version__)

2.8.2


#2.데이터 읽어오기

In [3]:
import os, re, glob 
import numpy as np
import tensorflow as tf



txt_file_path = '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/*'

txt_list = glob.glob(txt_file_path)

print(txt_list)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

['/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/alicia-keys.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/al-green.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/bjork.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/beatles.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/blink-182.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/bieber.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/amy-winehouse.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/britney-spears.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/bruce-springsteen.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/bruno-mars.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/bob-marley.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/disney.txt', '/content/drive/MyDrive/Aiffel_data/Exp6-composition/project/cake.t

#3.데이터 정제

In [4]:
for idx, sentence in enumerate(raw_corpus):

    print(sentence)
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Ooh....... New York x2 Grew up in a town that is famous as a place of movie scenes
Ooh....... New York x2 Grew up in a town that is famous as a place of movie scenes
Noise is always loud, there are sirens all around and the streets are mean
Noise is always loud, there are sirens all around and the streets are mean
If I can make it here, I can make it anywhere, that's what they say
If I can make it here, I can make it anywhere, that's what they say
Seeing my face in lights or my name on marquees found down on Broadway Even if it ain't all it seems, I got a pocket full of dreams
Seeing my face in lights or my name on marquees found down on Broadway Even if it ain't all it seems, I got a pocket full of dreams
Baby, I'm from New York
Baby, I'm from New York
Concrete jungle where dreams are made of
Concrete jungle where dreams are made of
There's nothing you can't do
There's nothing you can't do
Now you're in New York
Now you're in New York
These streets will make you feel brand new
These s

In [5]:
# 입력된 문장을
# 01. 양쪽 공백 제거
# 02. 여러개 공백 하나로 만들기
# 03. ()[] 제거 특수문자 제거
# 04. 모든 문자 대문자 
def preprocess_sentence(sentence):
    setentce = sentence.strip()
    sentence = re.sub('[" "]+', " ", sentence) 
    sentence = re.sub("[^a-zA-Z \'(\)\[\]]+", "", sentence) 
    sentence = re.sub("\[[\w\d\s-]*\]", "", sentence)
    sentence = re.sub("\([\w\d\s-]*\)", "", sentence)
    if (str.isupper(sentence) ==True):
      sentence = ""
    sentence = sentence.lower().strip()
    if len(sentence) != 0:
      sentence = '<start> ' + sentence + ' <end>' 

    sentence = sentence.lower().strip()

    return sentence




# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("UPPER NO UPPER"))

In [6]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == "+": continue
    if sentence[-1] == "-": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)

    if len(preprocessed_sentence) == 0: continue 
    if preprocessed_sentence == '<start> the queen of hearts <end>' : continue
    if len(preprocessed_sentence.split()) > 15 : continue    


    corpus.append(preprocessed_sentence)


        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

["<start> baby i'm from new york <end>",
 '<start> concrete jungle where dreams are made of <end>',
 "<start> there's nothing you can't do <end>",
 "<start> now you're in new york <end>",
 '<start> these streets will make you feel brand new <end>',
 '<start> big lights will inspire you <end>',
 '<start> such a melting pot on the corner selling rock preachers pray to god <end>',
 '<start> hail a gypsycab takes me down from harlem to the brooklyn bridge <end>',
 "<start> baby i'm from new york <end>",
 '<start> concrete jungle where dreams are made of <end>']

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다

def tokenize(corpus):
    # 12000개 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )

    # corpus를 이용해 tokenizer 내부의 '단어장'을 완성합니다
    tokenizer.fit_on_texts(corpus)

    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus) 

    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   44   18 ...    0    0    0]
 [   2 2836 2011 ...    0    0    0]
 [   2  162  174 ...    0    0    0]
 ...
 [   2    7   42 ...    0    0    0]
 [   2   97   86 ...    0    0    0]
 [   2    7   42 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa6d43a9cd0>


In [ ]:
# 단어 빈도수 
# for k, v in tokenizer.word_counts.items():
#   if len(k) == 1:
#     print(k, v)
# print(tokenizer.word_counts.items())

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다

# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  44  18  63 151 710   3   0   0   0   0   0   0   0]
[ 44  18  63 151 710   3   0   0   0   0   0   0   0   0]


#4.평가 데이터셋 분리

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [10]:
#enc_train, enc_val, dec_train, dec_val = # <코드 작성>

from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    )

print('enc_train 개수: ', len(enc_train),', enc_val 개수: ', len(enc_val))


enc_train 개수:  128212 , enc_val 개수:  32053


In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

#5.인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!



In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.

for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.04436796e-04, -2.93278972e-05, -9.07612775e-05, ...,
         -7.65805671e-05, -4.48094434e-05,  1.78937116e-04],
        [-2.51840684e-04, -8.65907423e-05, -1.40787932e-04, ...,
         -2.38995999e-04, -4.17903735e-04,  1.90494422e-04],
        [-5.48079726e-04, -3.28379829e-04, -2.12942381e-04, ...,
         -5.10018872e-05, -4.86177480e-04,  2.95930309e-04],
        ...,
        [-1.49172742e-03, -9.49848676e-04, -4.18400305e-04, ...,
          3.04138870e-04, -8.12267826e-04, -3.67788947e-04],
        [-1.70879776e-03, -1.35561905e-03, -4.58778173e-04, ...,
          3.79180623e-04, -1.08084606e-03, -4.20254975e-04],
        [-1.92951597e-03, -1.74995116e-03, -5.17118431e-04, ...,
          4.62063588e-04, -1.33457361e-03, -4.33335779e-04]],

       [[-1.04436796e-04, -2.93278972e-05, -9.07612775e-05, ...,
         -7.65805671e-05, -4.48094434e-05,  1.78937116e-04],
        [ 4.19863791e-05,  2.49771576e-04, -4

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  18882560  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  24590049  
                                                                 
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [15]:
#Loss
#모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!



optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)
model.compile(loss=loss, optimizer=optimizer)

model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))



Epoch 1/10
500/500 [==============================] - 137s 269ms/step - loss: 3.3773 - val_loss: 2.9795
Epoch 2/10
500/500 [==============================] - 134s 269ms/step - loss: 2.8124 - val_loss: 2.7312
Epoch 3/10
500/500 [==============================] - 134s 268ms/step - loss: 2.5394 - val_loss: 2.5660
Epoch 4/10
500/500 [==============================] - 134s 268ms/step - loss: 2.2716 - val_loss: 2.4367
Epoch 5/10
500/500 [==============================] - 134s 268ms/step - loss: 2.0146 - val_loss: 2.3370
Epoch 6/10
500/500 [==============================] - 134s 268ms/step - loss: 1.7765 - val_loss: 2.2679
Epoch 7/10
500/500 [==============================] - 134s 268ms/step - loss: 1.5656 - val_loss: 2.2186
Epoch 8/10
500/500 [==============================] - 134s 268ms/step - loss: 1.3882 - val_loss: 2.1939
Epoch 9/10
500/500 [==============================] - 134s 268ms/step - loss: 1.2459 - val_loss: 2.1792
Epoch 10/10
500/500 [==============================] - 134s 268m

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [24]:
generate_text(model, tokenizer, init_sentence="<start> you ", max_len=20)

'<start> you know that you need a rider <end> '


# 회고

### - 이번 프로젝트에서 **어려웠던 점**.
* 정규표현식 개념이 익숙하지 않아 데이터 정제시에 힘들었지만 새로 배우게 되어 좋은 기회였다.





### - 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.

**{알아낸 점}**
1. 정규표현식 사용 <정규표현식 유튜브 참고>
* https://youtu.be/t3M6toIflyQ
* https://youtu.be/dTDoTR0MXjU


    sentence = re.sub("[^a-zA-Z \'(\)\[\]]+", "", sentence) 
    sentence = re.sub("\[[\w\d\s-]*\]", "", sentence)
    sentence = re.sub("\([\w\d\s-]*\)", "", sentence)

2. split_train_test 을 응용해서 train _validation 셋으로 분리하기


    from sklearn.model_selection import train_test_split
    enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    )


3. validation loss 설정 하기 

:tensorflow fit 다큐먼트 검색

    model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))





### - 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.





>#### **루브릭평가 지표**
>|번호|평가문항|상세기준|
>|:---:|---|---|
>|1|  가사 텍스트 생성 모델이 정상적으로 동작한다. |텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되었다.|
>|2|데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었다.| 특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었다.|
>|3|텍스트 생성모델이 안정적으로 학습되었다.|텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌다.|

>  **hidden_size = 1024->2048 로 조정했습니다.**



### - **자기 다짐**

* 정규표현식 익숙해지게 공부해보기

### - **참고**

RNN Youtube : https://www.youtube.com/watch?v=NHb6jNH2YTg

Tensor : https://rekt77.tistory.com/102

RNN : https://nicola-ml.tistory.com/107

fit_on_texts : https://www.dinolabs.ai/186

Reg : https://blog.naver.com/ekqls9960/222488864305

Super() : https://rednooby.tistory.com/56

Tensor_Loss: https://hwiyong.tistory.com/335